In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report,accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import tensorflow as tf
# from xgboost import XGBClassifier

In [ ]:
#Loads in a dataframe with previous modelling results in if it exists.
try:
    metrics_df = pd.read_csv('Modeling_Results')
except:
    metrics_df = pd.DataFrame()
df_metrics.drop(columns=['Unnamed: 0', 'index'], inplace=True)

In [ ]:
#Returns a dictionary of metrics for each algorithm
def metrics_to_dict(algorithm, y_test, y_pred, train_acc, params):
    results = {}
    prec = metrics.precision_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_pred, y_test)
    recall = metrics.recall_score(y_test,y_pred)
    f1 = metrics.f1_score(y_test,y_pred)
    results['Algorithm'] = algorithm
    results['Train Accuracy'] = train_acc
    results['Hold Out Accuracy'] = accuracy
    results['Precision'] = prec
    results['Recall'] = recall
    results['F1 Score'] = f1
    results['Parameters'] = [params]
    
    return results

In [ ]:
#Reads in a dataset with te FFT info and sex of speaker
df_fft = pd.read_csv('full_dataset_sex')
df_fft.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

In [ ]:
df_fft.iloc[:,:190].columns

In [ ]:
#Takes the first 25 audio samples of each speaker and averages them.
df_mean_3 = df_fft.groupby('ID').head(25)
df_mean_3 = df_mean_3.groupby('ID').mean()

In [ ]:
#Averages out FFT for every speaker. Reduces the number of datapoints to 5000
mean_df = df_fft.groupby('ID').mean()

In [ ]:
#Trying out different numbers of PCA features
n=25
pca = PCA(n_components=n)
pca_X = pca.fit_transform(df_mean_3.iloc[:,:190])
df_features_PCA = pd.DataFrame(pca_X)
df_features_PCA.drop(columns = ['Unnamed: 0', 'Gender'], inplace=True)
pca.explained_variance_ratio_

In [ ]:
df_features_PCA.iloc[:,193:]

In [ ]:
#Setting the X and y variables. Tested using 1 second of FFT, PCA components, and mean of FFT data.
X = df_fft.iloc[:,:190]
y = df_fft['Sex']
# X = mean_df.iloc[:,:190]
# X = df_mean_3.iloc[:,:190]
# X = df_features_PCA
# y = df_mean_3['Sex']
X = scale(X)

In [ ]:
#Creating a Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
#Testing to make sure no bugs happened
len(X_train), len(y_train)

In [ ]:
#Logistic Regression with GridSearch of hyperparameters
log_params = {'C': [ 1e-3, 1e-2, 1e-1],
             'penalty': ['l1','l2']}
log = GridSearchCV(LogisticRegression(), log_params, scoring='accuracy', cv = 3)
log.fit(X_train,y_train)
y_scorelog=log.predict_proba(X_test)[:,1]
y_problog = log.predict_proba(X_test)
coefficients = list(log.best_estimator_.coef_)
fpr_log, tpr_log,_ = roc_curve(y_test, y_scorelog)
roc_auc = auc(fpr_log, tpr_log)
y_pred = log.predict(X_test)
log.score(X_train, y_train), log.score(X_test, y_test), log.best_params_

In [ ]:
log.best_estimator_.coef_

In [ ]:
#Saving the results to a dataframe
metrics_df = metrics_df.append(pd.DataFrame(metrics_to_dict('Logistic Regression', y_test, y_pred, log.score(X_train, y_train), log.best_params_)).reset_index())

In [ ]:
#Gradient boosting grid search of hyperparameters
gb_param = {'n_estimators' : [100,200],
        'max_depth' : [1,2],
        'learning_rate': [1e-2, 1e-1, 1]}
gradboost = GridSearchCV(GradientBoostingClassifier(), gb_param,scoring = 'accuracy', cv= 2)
gradboost.fit(X_train, y_train)
y_score_gboost=gradboost.predict_proba(X_test)[:,1]
y_prob_gboost = gradboost.predict_proba(X_test)
fpr_gboost, tpr_gboost,_ = roc_curve(y_test, y_score_gboost)
roc_auc = auc(fpr_gboost, tpr_gboost)
y_pred_g = gradboost.predict(X_test)
gradboost.score(X_train, y_train), gradboost.score(X_test, y_test), gradboost.best_params_

In [ ]:
#Trying my classmate Chris's voice
chris_fft = pd.read_csv('Chris_fft')
gradboost.predict(chris_fft.iloc[:,1:])

In [ ]:
#Adding to metrics dataframe
metrics_df = metrics_df.append(pd.DataFrame(metrics_to_dict('Gradient Boosting w/ mean', y_test, y_pred_g, gradboost.score(X_train, y_train), gradboost.best_params_)).reset_index())

In [ ]:
#Tried SVM with PCA components
svm_params ={'C' : [1e-1,1,1e1,11,12,13],
        'kernel' : ['rbf','linear'],
        'degree': [2,3]}
svm = GridSearchCV(SVC(), svm_params, scoring= 'accuracy', cv=3)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print(svm.score(X_train, y_train)), print(svm.score(X_test, y_test)), print(svm.best_params_)

In [ ]:
#Adding to results to dataframe
metrics_df = metrics_df.append(pd.DataFrame(metrics_to_dict('SVM(PCA 25 w/mean 10)', y_test, y_pred, svm.score(X_train, y_train), svm.best_params_)).reset_index())

In [ ]:
#Ordering the models and hyperparamaters that yielded the highest test accuracy
metrics_df.sort_values('Hold Out Accuracy', ascending=False)

In [ ]:
#Saving results dataframe
metrics_df.to_csv('Modeling_Results')

I tried some XGBOOST but it didn't perform better than gradient boosting models

In [ ]:
xgb_param = {'n_estimators' : [100,200,300],
        'max_depth' : [1,2,3,5],
        'learning_rate': [1e-2, 1e-1, 1]}
xgb = GridSearchCV(XGBClassifier(), xgb_param, cv=3)
xgb.fit(X_train,y_train)

In [ ]:
xgb = XGBClassifier(max_depth=1,learning_rate=0.1,n_estimators=380)
xgb.fit(X_train, y_train)

In [ ]:
y_pred_xgb = xgb.predict(X_test)

In [ ]:
metrics_df = metrics_df.append(pd.DataFrame(metrics_to_dict('XGBoost(PCA+8Features)', y_test, y_pred_xgb, xgb.score(X_train, y_train), 'None')).reset_index())